In [35]:
using Pkg
Pkg.add("NLsolve")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [36]:
using LinearAlgebra, NLsolve

In [38]:
global params=(
    4.2637, #A
    0.3,    #α
    0.8203, #β
    2.4394, #κ
    -0.058, #ρ
    0.04,   #ν
    0.0001, #θ
    0.1063) #χ

(4.2637, 0.3, 0.8203, 2.4394, -0.058, 0.04, 0.0001, 0.1063)

In [39]:
function output(K,Lf,Lr,E)
    A=params[1]
    α=params[2]
    κ=params[4]
    ρ=params[5]
    ν=params[6]
    χ=params[8]

    capital_share=K^α
    energy_share=(κ*Lf^ρ+Lr^ρ)^(ν/ρ)
    other_labour=(1-Lf-Lr)^(1-α-ν)
    damage=ℯ^(-χ*E)
    
    Y=A*capital_share*energy_share*other_labour*damage
    return Y
end

output (generic function with 1 method)

In [40]:
#test
#output(0.2461,0.03995,0.0172,0.0)

In [41]:
function consumption(Y)
    α=params[2]
    β=params[3]

    C=(1-α*β)*Y
    return C
end

consumption (generic function with 1 method)

In [42]:
#test
#consumption(1.0)

In [43]:
function capital_plus(Y,C)
    Kplus=Y-C
    return Kplus
end

capital_plus (generic function with 1 method)

In [44]:
#test
#capital_plus(1.0,0.7539)

In [45]:
function fossil_labour()
    α=params[2]
    κ=params[4]
    ρ=params[5]
    ν=params[6]

    Lf=ν/((1-α)*(1+κ^(1/(ρ-1))))
    return Lf
end

fossil_labour (generic function with 1 method)

In [46]:
function emissions_plus(E,Lf)

    e_plus=E+Lf
    return e_plus
end

emissions_plus (generic function with 1 method)

In [47]:
#test
#fossil_labour()

In [48]:
function renew_labour(Lf)
    κ=params[4]
    ρ=params[5]

    Lr=Lf*(κ^(1/(ρ-1)))
    return Lr
end

renew_labour (generic function with 1 method)

In [49]:
#test
#renew_labour(fossil_labour())

In [50]:
function mpl(Y,Lf,Lr)
    α=params[2]
    ν=params[6]

    MPL=(1-α-ν)*(Y/(1-Lf-Lr))
    return MPL
end

mpl (generic function with 1 method)

In [51]:
#test
#mpl(1.0,0.03995,0.0172)

In [52]:
function mpl_f(Y,Lf,Lr)
    κ=params[4]
    ρ=params[5]
    ν=params[6]

    MPL_f=ν*κ*(Lf^(ρ-1))*(Y/(κ*(Lf^ρ)+(Lr^ρ)))
    return MPL_f
end

mpl_f (generic function with 1 method)

In [53]:
#test
#mpl_f(1.0,0.03995,0.0172)

In [54]:
function mpl_r(Y,Lf,Lr)
    κ=params[4]
    ρ=params[5]
    ν=params[6]

    MPL_r=ν*(Lr^(ρ-1))*(Y/(κ*(Lf^ρ)+(Lr^ρ)))
    return MPL_r
end

mpl_r (generic function with 1 method)

In [55]:
#test
#mpl_r(1.0,0.03995,0.0172)

In [22]:
function tau(Y)
    β=params[3]
    χ=params[8]

    τ=Y*(β*χ/(1-β))
    return τ
end

tau (generic function with 1 method)

In [23]:
#test
#tau(0.9968)

In [24]:
function mpl_fτ(Y,Lf,Lr)
    
    MPL_fτ=mpl_f(Y,Lf,Lr)-tau(Y)
    return MPL_fτ
end

mpl_fτ (generic function with 1 method)

In [25]:
#test
#mpl_fτ(1.0,0.03995,0.0172)

In [26]:
function mpl_eq(Y,Lf,Lr)

    MPL_eq=(mpl(Y,Lf,Lr)-mpl_fτ(Y,Lf,Lr))^2+(mpl(Y,Lf,Lr)-mpl_r(Y,Lf,Lr))^2
    return MPL_eq
end

mpl_eq (generic function with 1 method)

In [27]:
#test
#mpl_eq(1,0.03995,0.0172)

In [109]:
global data_0=[1.0,0.0,fossil_labour(),renew_labour(fossil_labour())] #Y,E,Lf,Lr

4-element Vector{Float64}:
 1.0
 0.0
 0.03994676921142965
 0.017196087931427493

In [96]:
#typeof(data_0) #test

Vector{Float64} (alias for Array{Float64, 1})

In [111]:
n=13
var=data_0

function f(x)

    K=capital_plus(var[1],consumption(var[1]))
    E=emissions_plus(var[2],var[3])
    Y=output(K,x[1],x[2],E)
    
    f=mpl_eq(Y,x[1],x[2])
    return f
end

for i in 1:n
    sol=nlsolve(f,[var[3],var[4]])
    s=sol.zero
    
    Kplus=capital_plus(var[1],consumption(var[1]))
    Eplus=emissions_plus(var[2],s[1])
    var[1]=output(Kplus,s[1],s[2],Eplus) #Y
    var[2]=Eplus #E
    var[3]=s[1] #Lf
    var[4]=s[2] #Lr

    a=round(var[1]*100,digits=2)
    b=round(var[2],digits=4)
    c=round(var[3],digits=4)
    d=round(var[4],digits=4)
    println("t=$i, Y=$a%, E=$b, Lf=$c, Lr=$d")

end

t=1, Y=99.42%, E=0.024, Lf=0.024, Lr=0.0171
t=2, Y=99.0%, E=0.0481, Lf=0.024, Lr=0.0171
t=3, Y=98.62%, E=0.0721, Lf=0.024, Lr=0.0171
t=4, Y=98.25%, E=0.0961, Lf=0.024, Lr=0.0171
t=5, Y=97.89%, E=0.1202, Lf=0.024, Lr=0.0171
t=6, Y=97.53%, E=0.1442, Lf=0.024, Lr=0.0171
t=7, Y=97.18%, E=0.1682, Lf=0.024, Lr=0.0171
t=8, Y=96.82%, E=0.1923, Lf=0.024, Lr=0.0171
t=9, Y=96.47%, E=0.2163, Lf=0.024, Lr=0.0171
t=10, Y=96.12%, E=0.2404, Lf=0.024, Lr=0.0171
t=11, Y=95.77%, E=0.2644, Lf=0.024, Lr=0.0171
t=12, Y=95.42%, E=0.2884, Lf=0.024, Lr=0.0171
t=13, Y=95.07%, E=0.3125, Lf=0.024, Lr=0.0171
